In [46]:
# imports
from owlready2 import *

# Create ontology

In [47]:
# create new ontology
game_ontology = get_ontology("http://test.org/game_ontology.owl")
game_ontology.destroy()
game_ontology = get_ontology("http://test.org/game_ontology.owl")

# Import existing ontology

In [48]:
# load existing video game ontology 
if not onto_path:
    onto_path.append("..")
video_game_ontology = get_ontology(f"file://../video_game_ontology.owl#").load()

# importing loaded video game ontology into our game ontology
if not game_ontology.imported_ontologies:
    game_ontology.imported_ontologies.append(video_game_ontology)

print(list(video_game_ontology.classes()))

[owl.Thing, video_game_ontology.Game, 0.1.Agent, video_game_ontology.Achievement, video_game_ontology.Character, video_game_ontology.Feature, video_game_ontology.Item, video_game_ontology.Genre, video_game_ontology.Leaderboard, video_game_ontology.PlayingArea, video_game_ontology.Session, video_game_ontology.Player, video_game_ontology.InstantaneousEvent, ontology.PopulatedPlace, video_game_ontology.GameProduct, video_game_ontology.InAppPurchaseEvent, v1.ProductOrService, video_game_ontology.Collection, video_game_ontology.Completion, video_game_ontology.Curiosity, video_game_ontology.Fandom, video_game_ontology.GainEvent, video_game_ontology.GameEvent, schema.org.Product, video_game_ontology.HardMode, video_game_ontology.LoseEvent, video_game_ontology.Loyalty, video_game_ontology.Luck, video_game_ontology.MenuEvent, video_game_ontology.Minigame, video_game_ontology.Multiplayer, video_game_ontology.Paragon, video_game_ontology.PlayerEvent, video_game_ontology.SpecialPlayStyle, video_ga

## Atomic concepts (classes)

In [49]:
with game_ontology:
    class Game(Thing):
        pass
    class Equipment(Thing):
        pass
    class Field(Thing):
        pass    
    class Spectator(Thing):
        pass
    # is it atomic??
    class Card(Equipment):
        pass

## Properties

In [50]:
with game_ontology:
    class require(Game >> Equipment):
        pass
    class required_by(Equipment >> Game):
        inverse_property = require
        pass
    class watch(Spectator >> Game):
        pass
    class watched_by(Game >> Spectator):
        inverse_property = watch
        pass
    class played_on(Game >> Field):
        pass
    class has_players_number(Game >> int, FunctionalProperty):
        pass
    class has_greater_value_than(Card >> Card, TransitiveProperty):
        pass
    class has_less_value_than(Card >> Card, TransitiveProperty):
        inverse_property = has_greater_value_than
        pass
    class has_team(Game >> bool, FunctionalProperty):
        pass
    class is_professional(Game >> bool, FunctionalProperty):
        pass
    class is_amateur(Game >> bool, FunctionalProperty):
        Not(is_professional)
        pass
    class has_interest_in(Spectator >> Game):
        pass
    class has_to_remain_silent(Spectator >> bool, FunctionalProperty):
        pass

## Composed concepts (classes)

In [51]:
with game_ontology:    
    class MultiPlayerGame(Game):
        equivalent_to = [Game
                        & has_players_number.min(2)]        
        pass
    class SinglePlayerGame(Game):
        equivalent_to = [Game
                        & has_players_number.exactly(1)] 
        pass
    class TeamGame(Game):
        equivalent_to = [MultiPlayerGame
                        & has_team.only(True)]      
        pass
    class PhysicalActivityGame(Game):
        pass
    class IntelectualGame(Game):
        pass
    class SportEquipment(Equipment):
        pass
    class Table(Equipment):
        pass
    class Dice(Equipment):
        pass
    class Paper(Equipment):
        pass
    class Pencil(Equipment):
        pass
    class Computer(Equipment):
        pass
    class Ball(Equipment):
        pass
    class Lawn(Field):
        pass
    class Board(Field):
        pass
    class Fan(Spectator):
        equivalent_to = [Spectator
                        & has_interest_in.only(Game)]
        pass
    class Kibitzer(Spectator):
        equivalent_to = [Spectator
                        & has_to_remain_silent.only(True)]
        pass
    class SportGame(PhysicalActivityGame):
        equivalent_to = [PhysicalActivityGame
                        & require.some(SportEquipment)
                        & watched_by.some(Spectator)
                        & is_professional.some(True)]
        pass
    class LawnGame(SportGame):
        equivalent_to = [SportGame
                        & played_on.only(Lawn)]
        pass
    class BallGame(SportGame):
        equivalent_to = [SportGame
                        & require.only(Ball)]
        pass
    class TeamSportGame(TeamGame, SportGame):
        pass
    class IndividualSportGame(SinglePlayerGame, SportGame):
        pass
    class LogicalGame(IntelectualGame):
        pass
    class StrategyGame(IntelectualGame):
        pass
    class KnowledgeGame(IntelectualGame):
        pass
    class TableTopGame(LogicalGame, StrategyGame):
        equivalent_to = [LogicalGame
                        & StrategyGame
                        & (MultiPlayerGame | SinglePlayerGame)
                        & require.only(Table)
                        & is_professional.some(True)]
        pass
    class BoardGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & played_on.only(Board)]
        pass
    class CardGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Card)]
        pass
    class DiceGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Dice)]
        pass
    class PaperAndPencilGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Paper)
                        & require.only(Pencil)]
        pass
    class VideoGame(Game):
        is_a = [video_game_ontology.Game]
        equivalent_to = [Game
                        & require.only(Computer)
                        & is_professional.some(True)]


# Instances

In [52]:
horseshoes = LawnGame("horseshoes")
croquet = LawnGame("croquet")

football = BallGame("football", has_team = True, is_professional = True)
basketball = BallGame("basketball", has_team = True, is_professional = True)
volleyball = BallGame("volleyball", has_team = True, is_professional = True)

chess = BoardGame("chess", has_players_number = 2)
go = BoardGame("go", has_players_number = 2)

bridge = CardGame("bridge", has_players_number = 4)
poker = CardGame("poker")

poker_dice = DiceGame("poker_dice")

tic_tac_toe = PaperAndPencilGame("tic_tac_toe", has_players_number = 2)
sudoku = PaperAndPencilGame("sudoku")

pong = VideoGame("pong")
minecraft = VideoGame("minecraft")
super_mario = VideoGame("super_mario", has_players_number = 1, is_amateur = True)

jack = Card("jack")
queen = Card("queen", has_greater_value_than = [jack])
king = Card("king", has_greater_value_than = [queen])

## Tests

In [53]:
print(king.has_greater_value_than)
print(king.INDIRECT_has_greater_value_than)
print(jack.INDIRECT_has_less_value_than)

# prints list of all superclasses
print(TeamGame.is_a)

[game_ontology.queen]
[game_ontology.jack, game_ontology.queen]
[game_ontology.queen, game_ontology.king]
[game_ontology.Game]


## Save

In [54]:
game_ontology.save("../game_ontology.owl")